# Investigate ```mercantile``` for creating global mapping grid

We want to create a sampling grid that is compatable with the standard used by Google in Sperical Web Mercator projection. This will allow simple compatability with external tools and web tile servers (almost all map services use this system now).

The tiling system is based on a scaled pyramid that sub-divides each tile in the higher level into 4 square components. Zoom Level 10 offers tiles approximately equivalent to the current FloodMapper tiles. Zoom level 9 (x4 times larger) or 8 (16x larger) may be a good size to perform the vectorisation step.

Web Mercator projection is encoded as [EPSG:4326](https://epsg.io/4326) for angular definitions [EPSG:3857](https://epsg.io/3857) for projected distances. 

However, conversions to EPSG:3857 result in Northing errors up to 43 km (!!!) at high latitudes and up to 0.7% errors in scale. For detailed work requiring accurate scales and distances, we should convert to the local [UTM region](https://www.usgs.gov/faqs/what-does-term-utm-mean-utm-better-or-more-accurate-latitudelongitude). For most mapping steps, we will work in angular scales, converting to a projected system at the last possible time.

The Sydney UTM Zone is 56S = [EPSG:32756](https://epsg.io/32756).

For Australia, one of the [GDA2020 / MGA Zones](https://www.spatial.nsw.gov.au/__data/assets/pdf_file/0020/216614/2017_Janssen_APAS2017_GDA2020-AUSGeoid2020-ATRF_introduction.pdf) could also be used (e.g., MGA Zone 55 = [EPSG:7855](https://epsg.io/7855) for central NSW).


## Links and resources

**Core Refs**
* https://www.maptiler.com/google-maps-coordinates-tile-bounds-projection/#1/161.35/-2.53
* https://mercantile.readthedocs.io/en/stable/api/modules.html
* https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames (Filename(url) format is /zoom/x/y.png)
* https://en.wikipedia.org/wiki/Web_Mercator_projection
* https://qldglobe.information.qld.gov.au/help-info/mapping-fundamentals.html
* https://source.opennews.org/articles/choosing-right-map-projection/


**Questions**
* https://gis.stackexchange.com/questions/437144/how-to-create-raster-tiles-in-the-slippy-map-tilenames-format-from-aerial-imager
* https://python.hotexamples.com/examples/mercantile/-/tiles/python-tiles-function-examples.html
* https://medium.com/helpful-human/world-adventures-how-i-learned-all-about-slippy-tile-maps-28cd84a89e12
* https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.buffer.html

In [ ]:
# Necessary imports
import os
os.environ['USE_PYGEOS'] = '0'
import numpy as np
import math as math
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import mercantile
import folium
import matplotlib.pyplot as plt

## Defining a grid for Australia

In [ ]:
# Initial bounding box covering all of Australia
# (long_min, lat_min, long_max, lat_max)
# (west, south, east, north)
#bounds_initial = (112.900000000000, -44.00516044138397, 
#                  153.63872785102905, -10.244936010554465)

# Test for larger areas
#bounds_initial = (-180.0, -10.0, 
#                  180.0, 10.0)

# Test for high latitudes
bounds_initial = (150.6, -85.0, 
                  151.6, 0.0)

# Test for northern latitudes
#bounds_initial = (-22, 0.0, 
#                  -20, 85.0)

In [ ]:
# Set the coordinates
west = bounds_initial[0]
south = bounds_initial[1]
east = bounds_initial[2]
north = bounds_initial[3]

# Grab lists of tiles at different zoom levels within the box
tiles_10 = list(mercantile.tiles(west, south, east, north, 10))
tiles_9 = list(mercantile.tiles(west, south, east, north, 9))
tiles_8 = list(mercantile.tiles(west, south, east, north, 8))
tiles_7 = list(mercantile.tiles(west, south, east, north, 7))
tiles_6 = list(mercantile.tiles(west, south, east, north, 6))
tiles_5 = list(mercantile.tiles(west, south, east, north, 5))

print(f"There are {len(tiles_10)} tiles_10.")
print(f"There are {len(tiles_9)} tiles_9.")
print(f"There are {len(tiles_8)} tiles_8.")

In [ ]:
# TEST: Grab the unique quadkey that maps the location in the pyramid
tile = tiles_10[-1]
quadkey = mercantile.quadkey(tile)
quadkey

In [ ]:
l = mercantile.lnglat(tile.x, tile.y)
np.degrees(l.lat)

In [ ]:
# TEST: Generate a feature dictionary that can be used to make a GeoJSON
fd = mercantile.feature(tile)
fd

In [ ]:
def custom_feature(tile):
    """
    Generate extra features for use with FloodMapper.
    
    Parameters:
    - tile: A mercantile Tile object.

    Returns:
    - features: A dictionary of features.
    
    """
    
    # Create the default dictionary and add more features
    fd = mercantile.feature(
        tile, 
        props={'quadkey': mercantile.quadkey(tile), 
               'patch_name': f"G_{tile.z}_{tile.x}_{tile.y}",
               'zoom': tile.z})

    # Add a centroid feature
    cent_x, cent_y = np.mean(fd['geometry']['coordinates'], axis=1)[0]
    fd['properties']['cent_x'] = cent_x
    fd['properties']['cent_y'] = cent_y

    # Calculate the Cos(lat) factor
    cos_factor = math.cos(math.radians(cent_y))
    fd['properties']['cos_factor'] = cos_factor

    return fd


# Generate GeoDataFrames from the features
crs_code = "EPSG:4326"
features_10 = [custom_feature(tile) for tile in tiles_10] 
grid_10 = gpd.GeoDataFrame.from_features(
    features_10, crs=crs_code, 
    columns=["geometry", "patch_name", "quadkey", "zoom", "cent_x", "cent_y", "cos_factor"])
features_9 = [custom_feature(tile) for tile in tiles_9] 
grid_9 = gpd.GeoDataFrame.from_features(
    features_9, crs=crs_code, 
    columns=["geometry", "patch_name", "quadkey", "zoom", "cent_x", "cent_y", "cos_factor"])
features_8 = [custom_feature(tile) for tile in tiles_8] 
grid_8 = gpd.GeoDataFrame.from_features(
    features_8, crs=crs_code, 
    columns=["geometry", "patch_name", "quadkey", "zoom", "cent_x", "cent_y", "cos_factor"])
features_7 = [custom_feature(tile) for tile in tiles_7] 
grid_7 = gpd.GeoDataFrame.from_features(
    features_7, crs=crs_code, 
    columns=["geometry", "patch_name", "quadkey", "zoom", "cent_x", "cent_y", "cos_factor"])
features_6 = [custom_feature(tile) for tile in tiles_6] 
grid_6 = gpd.GeoDataFrame.from_features(
    features_6, crs=crs_code, 
    columns=["geometry", "patch_name", "quadkey", "zoom", "cent_x", "cent_y", "cos_factor"])
features_5 = [custom_feature(tile) for tile in tiles_5] 
grid_5 = gpd.GeoDataFrame.from_features(
    features_5, crs=crs_code, 
    columns=["geometry", "patch_name", "quadkey", "zoom", "cent_x", "cent_y", "cos_factor"])

In [ ]:
# Plot the grid on openstreet maps
m = grid_10.explore(style_kwds={"fillOpacity": 0.1,}, name="Zoom 10", highlight=False)
grid_9.explore(m=m, style_kwds={"fillOpacity": 0.0, "color": "red"}, name="Zoom 9", highlight=False)
grid_8.explore(m=m, style_kwds={"fillOpacity": 0.0, "color": "green"}, name="Zoom 8", highlight=False)
folium.LayerControl(collapsed=False).add_to(m)

In [ ]:
m

## Choosing a pixel scale for Sentinel-2 imagery

In [ ]:
# Convert the CRS to a projected system
# EPSG:3857 ... inaccurate 
# EPSG:7855 ... better
# EPSG:32755 (WGS 84 / UTM zone 55S) ... best for central NSW 
#target_epsg = 3857
#target_epsg = 7855
#target_epsg = 32755
target_epsg=32756 # Sydney UTM Zone

grid_10_projected = grid_10.to_crs(epsg=target_epsg, inplace=False)
m = grid_10_projected.explore(name="Projected")
grid_10.explore(m=m, color="red", name="Angular")
folium.LayerControl(collapsed=False).add_to(m)

In [ ]:
m

In [ ]:
# Select the patches at extreme latitudes
#patch_names = ["G_8_231_128", "G_8_231_255"]
patch_names = ["G_10_941_1022", "G_10_941_512"]
#patch_names = ["G_10_452_511", "G_10_452_1"]
pe = grid_10_projected[grid_10_projected.patch_name.isin(patch_names)]
pe

In [ ]:
# Select and plot the upper patch
coords_x, coords_y = pe.iloc[0].geometry.boundary.coords.xy
print(f"X = {coords_x}")
print(f"Y = {coords_y}")
fig, ax = plt.subplots(1,1, figsize=(15,5))
ax.plot(coords_x, coords_y, color="red")
ax.scatter(coords_x[0], coords_y[0])
ax.set_xlabel("X Coordinate (m)")
ax.set_ylabel("Y Coordinate (m)")

In [ ]:
# Calculate lengths
print("UPPER PATCH")
u_south_side_m = coords_x[3] - coords_x[0]
u_north_side_m = coords_x[2] - coords_x[1]
print("Length south {:f} (km)".format(u_south_side_m/1000))
print("Length north {:f} (km)".format(u_north_side_m/1000))
u_west_side_m = coords_y[1] - coords_y[0]
u_east_side_m = coords_y[2] - coords_y[3]
print("Length west {:f} (km)".format(u_west_side_m/1000))
print("Length east {:f} (km)".format(u_east_side_m/1000))

In [ ]:
# Select and plot the lower patch
coords_x, coords_y = pe.iloc[1].geometry.boundary.coords.xy
print(f"X = {coords_x}")
print(f"Y = {coords_y}")
fig, ax = plt.subplots(1,1, figsize=(15,5))
ax.plot(coords_x, coords_y, color="red")
ax.scatter(coords_x[0], coords_y[0])
ax.set_xlabel("X Coordinate (m)")
ax.set_ylabel("Y Coordinate (m)")

In [ ]:
# Calculate lengths
print("UPPER PATCH")
l_south_side_m = coords_x[3] - coords_x[0]
l_north_side_m = coords_x[2] - coords_x[1]
print("Length south {:f} (km)".format(l_south_side_m/1000))
print("Length north {:f} (km)".format(l_north_side_m/1000))
l_west_side_m = coords_y[1] - coords_y[0]
l_east_side_m = coords_y[2] - coords_y[3]
print("Length west {:f} (km)".format(l_west_side_m/1000))
print("Length east {:f} (km)".format(l_east_side_m/1000))

In [ ]:
# SANITY CHECK: Cos(latitude) scaling factor
scale_factor = np.cos(np.radians(bounds_initial[1]))
print(scale_factor)
lower_len = u_north_side_m * scale_factor
print(f"CHANGE: Upper {u_north_side_m/1000} -> {lower_len/1000}")

**Pixel Size**

The maximum width width of a patch at low latitude is 39122m and we desire 10m pixel sizes. This means we need to sample the Zoom-10 patches with 3912 ~ 4000 pixels. This will be our base resolution.

At full resolution, Zoom-9 images will be 8000 pixels wide and Zoom-8 images will be 16000 pixels wide.

However, all patches will need to be padded on all sides to ensure overlap when mosaicing. In the older version of floodmapper the side lengths were expanded by 5% (2.5% on each side).



## Measure the patch width in degrees and calculate padding

In [ ]:
pe = grid_10[grid_10.patch_name.isin(patch_names)]
pe

In [ ]:
# Select and plot the upper patch
coords_x, coords_y = pe.iloc[1].geometry.boundary.coords.xy
print(f"X = {coords_x}")
print(f"Y = {coords_y}")
fig, ax = plt.subplots(1,1, figsize=(15,5))
ax.plot(coords_x, coords_y, color="red")
ax.scatter(coords_x[0], coords_y[0])
ax.set_xlabel("Longitude (deg)")
ax.set_ylabel("Latitude (deg)")

In [ ]:
# Calculate lengths
print("UPPER PATCH")
u_south_side_deg = coords_x[3] - coords_x[0]
u_north_side_deg = coords_x[2] - coords_x[1]
print("Length south {:f} (deg)".format(u_south_side_deg))
print("Length north {:f} (deg)".format(u_north_side_deg))
u_west_side_deg = coords_y[1] - coords_y[0]
u_east_side_deg = coords_y[2] - coords_y[3]
print("Length west {:f} (deg)".format(u_west_side_deg))
print("Length east {:f} (deg)".format(u_east_side_deg))

In [ ]:
# Calculate the padding in degrees
pad = u_north_side_deg*0.05/2
print(f"PADDING: {pad} (deg).")

The conversion factor between deg and m is (39121.412 m / 0.351562 deg) = 111278.841 at low lat.

A constant padding of 0.009 deg = 1001.5 m at the equator and 87m at -85 degrees latitude.

## Investigate buffering

In [ ]:
# Use Shapely buffer() to add padding to the tiles
grid_10_buffer = grid_10.buffer(0.009, join_style=2)

In [ ]:
# Plot the new tiles
m = grid_10_buffer.explore(name="Buffered", style_kwds={"fillOpacity": 0.1,})
grid_10.explore(m=m, color="red", name="Butted", style_kwds={"fillOpacity": 0.1,})
folium.LayerControl(collapsed=False).add_to(m)
m

Buffering using Shapely in angular-space does not work well as we need add different amounts to the X and Y sides.

## Investigating area preservation

* Slippy tiles become very narrow (E-W) and tall (N-S) at increasing latitudes.
* What do square tiles look like if placed at the centroids of the Zoom-10 slippy grid?

In [ ]:
# Function to create a projected 'square' patch
def mk_sq(row, side_deg=0.351562):
    
    x = []
    y = []
    delta = abs(side_deg/2.0)

    ## Create the tile polygon assuming equal side lengths (approx equal area)
    # Corner 0
    x.append( row.cent_x - delta/row.cos_factor)
    y.append( row.cent_y - delta)
    # Corner 1
    x.append( row.cent_x - delta/row.cos_factor)
    y.append( row.cent_y + delta)
    # Corner 2
    x.append( row.cent_x + delta/row.cos_factor)
    y.append( row.cent_y + delta)
    # Corner 3
    x.append( row.cent_x + delta/row.cos_factor)
    y.append( row.cent_y - delta)
    # Close the polygon
    x.append(x[0])
    y.append(y[0])
    
    xy_geom = Polygon(zip(x, y))
    
    return xy_geom
    
# Create a grid of equal area tiles
grid_10_ea = grid_10[["patch_name", "cent_x", "cent_y", "cos_factor"]].copy()
for indx, row in grid_10_ea.iterrows():
    grid_10_ea.loc[indx, 'geometry'] = mk_sq(row)
grid_10_ea = gpd.GeoDataFrame(grid_10_ea, crs=grid_10.crs)
grid_10_ea

In [ ]:
# Plot the tiles on a map
m = grid_10_ea.explore(style_kwds={"fillOpacity": 0}, name="Equal_Area")
grid_10.explore(m=m, color="red", name="Slippy Tiles", style_kwds={"fillOpacity": 0.1,})
folium.LayerControl(collapsed=False).add_to(m)

In [ ]:
m

## Investigate the change in side length and area with Latitude

We want to plot the change in side-length and area as a function of latitude. This will help inform how we want to generate tiles at different latitudes. The current idea is to decrease zoom levels in steps as we get to higher latitudes, so as to keep the tile area similar, within some bounds.

In [ ]:
# Add an area column to the slippy tile grid
grid_10_projected["area_m2"] = grid_10_projected.area
grid_10_projected["area_factor"] =  grid_10_projected.iloc[0]["area_m2"] / grid_10_projected["area_m2"]

# Add side length columns
def calc_north_side(x):
    coords_x, coords_y = x.boundary.coords.xy
    len_north_side = coords_x[2] - coords_x[1]
    return len_north_side
def calc_east_side(x):
    coords_x, coords_y = x.boundary.coords.xy
    len_east_side = coords_y[2] - coords_y[3]
    return len_east_side
grid_10_projected["side_north"] = grid_10_projected.geometry.apply(calc_north_side)
grid_10_projected["side_east"] = grid_10_projected.geometry.apply(calc_east_side)
grid_10_projected.head(2)

In [ ]:
# Plot the side-length as a function of latitude
plt.scatter(x=abs(grid_10_projected.cent_y), y=grid_10_projected.side_east/1000, s=1, color="r", alpha=0.2)
plt.scatter(x=abs(grid_10_projected.cent_y), y=grid_10_projected.side_north/1000, s=1, color="b", alpha=0.2)
plt.xlabel("Latitude (deg)")
plt.ylabel("Side-Length (km)")

The tiles maintain approximately the same aspect ratio with increasing latitude, meaning the longitude extent shrinks away from the Equator.

In [ ]:
# Plot the side-length as a function of latitude
plt.scatter(x=abs(grid_10_projected.cent_y), 
            y=grid_10_projected.side_east/grid_10_projected.side_north, 
            s=1, color="r", alpha=0.2)
plt.xlabel("Latitude (deg)")
plt.ylabel("Side Length Ratio")

There is a slight variation in aspect ratio across the latitude range, but nothing to worry about.

In [ ]:
# Plot the tile area vs latitude
ax = plt.scatter(x=abs(grid_10_projected.cent_y), y=grid_10_projected.area_m2/1e6, s=1)
plt.xlabel("Latitude (deg)")
plt.ylabel("Tile Area (km$^2$)")

In [ ]:
# Plot the change in area from the Equator to high latitude
ax = plt.scatter(x=abs(grid_10_projected.cent_y), y=grid_10_projected.area_factor, s=1, color="red", zorder=2)
plt.axhline(y=1, color="k", zorder=1, lw=0.5, ls="-")
plt.axhline(y=4, color="grey", zorder=1, lw=0.5, ls="--")
plt.axhline(y=16, color="grey", zorder=1, lw=0.5, ls="--")
plt.axhline(y=64, color="grey", zorder=1, lw=0.5, ls="--")
plt.axvline(x=75.8, color="grey", zorder=1, lw=0.5, ls="--")
plt.axvline(x=60.2, color="grey", zorder=1, lw=0.5, ls="--")
plt.axvline(x=82.8, color="grey", zorder=1, lw=0.5, ls="--")
plt.ylim(-1,20)
plt.xlabel("Latitude (deg)")
plt.ylabel("Area Shrink Factor")

**Ideas on Tiling Scheme**

Zoom-10 slippy-map tiles will become 4x smaller in area at 60 degrees latitude and 16x smaller at 75.8 degrees. We should decrease the Zoom level to 9 and 8, respectively before these breakpoints.

In [ ]:
m = grid_10_projected.explore(style_kwds={"fillOpacity": 0}, name="Zoom 10")
grid_9.explore(m=m, style_kwds={"fillOpacity": 0.0, "color": "black"}, name="Zoom 9", highlight=False)
grid_8.explore(m=m, style_kwds={"fillOpacity": 0.0, "color": "yellow"}, name="Zoom 8", highlight=False)
grid_7.explore(m=m, style_kwds={"fillOpacity": 0.0, "color": "magenta"}, name="Zoom 7", highlight=False)
grid_6.explore(m=m, style_kwds={"fillOpacity": 0.0, "color": "green"}, name="Zoom 6", highlight=False)
grid_5.explore(m=m, style_kwds={"fillOpacity": 0.0, "color": "red"}, name="Zoom 5", highlight=False)
folium.LayerControl(collapsed=False).add_to(m)

In [ ]:
m

In [ ]:
# Get the lower bounds of the last Zoom 10 tile before -60 deg lat.
transition_tilename = "G_10_940_727"
tt = grid_10[grid_10.patch_name==transition_tilename]
coords_x, coords_y = tt.iloc[0].geometry.boundary.coords.xy
print(f"X = {coords_x}")
print(f"Y = {coords_y}")
fig, ax = plt.subplots(1,1, figsize=(15,5))
ax.plot(coords_x, coords_y, color="red")
ax.scatter(coords_x[0], coords_y[0])
ax.set_xlabel("X Coordinate (m)")
ax.set_ylabel("Y Coordinate (m)")

**The transition latitude Zoom 10 -> Zoom 9 is +/-60.239811169998916**

In [ ]:
# Get the lower bounds of the last Zoom 10 tile before -60 deg lat.
transition_tilename = "G_10_940_847"
tt = grid_10[grid_10.patch_name==transition_tilename]
coords_x, coords_y = tt.iloc[0].geometry.boundary.coords.xy
print(f"X = {coords_x}")
print(f"Y = {coords_y}")
fig, ax = plt.subplots(1,1, figsize=(15,5))
ax.plot(coords_x, coords_y, color="red")
ax.scatter(coords_x[0], coords_y[0])
ax.set_xlabel("X Coordinate (m)")
ax.set_ylabel("Y Coordinate (m)")

**The transition latitude Zoom 9 -> Zoom 8 is +/-75.49715731893085**